In [1]:
import string
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import torch
from torch.autograd import Variable
import numpy as np
import torch.functional as F
import torch.nn.functional as F
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [2]:
def preprocess(sentence): 
    wordnet_lemmatizer = WordNetLemmatizer()
    sentence = stemmer.stem(sentence) #remove endings
    sentence = sentence.lower() #for writing from small litter
    tokenizer = RegexpTokenizer(r'\w+') #  #for removing puntuations
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [wordnet_lemmatizer.lemmatize(word) for word in tokens]
    filtered_words = [w for w in tokens if not w in stopwords.words('english')] #remove stop words
    filtered_words = [c for c in filtered_words if not c.isdigit()] #remove numbers
    return " ".join(filtered_words)

In [3]:
url="bbc.csv"
dataset = pd.read_csv(url)
dataset = dataset[dataset.category == 'tech']
dataset.index = np.arange(len(dataset))
dataset = dataset[:1]
print(dataset.head())
print(len(dataset))

  category                                               text
0     tech  tv future in the hands of viewers with home th...
1


In [4]:
corpus = dataset['text']
print(corpus)

0    tv future in the hands of viewers with home th...
Name: text, dtype: object


In [5]:
tokenized_corpus = []
stemmer = PorterStemmer()
for i in range(len(corpus)):
    sentence = preprocess(corpus[i])
    tokenized_corpus.append(word_tokenize((sentence)))

In [6]:
vocabulary = []
for sentence in tokenized_corpus:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)

In [7]:
len(word2idx)

275

In [8]:
window_size = 1
idx_pairs = []
# for each sentence
for sentence in tokenized_corpus:
    indices = [word2idx[word] for word in sentence]
    # for each word, threated as center word
    for center_word_pos in range(len(indices)):
        # for each window position
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make soure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

idx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array

In [9]:
#1 Создаем пустую матрицу 
array_occurance = [[0 for j in range(len(vocabulary))] for i in range(len(vocabulary))]

In [10]:
for i in range(len(vocabulary)):
    for j in range(len(vocabulary)):
        for m in range (len(idx_pairs)):
            if (i != j):
                if (idx_pairs[i][0] == idx_pairs[m][0]) and (idx_pairs[j][1] == idx_pairs[m][1]):
                    array_occurance[i][j] = array_occurance[i][j] + 1

In [11]:
source = []
target = []
weight = []
vocabulary_new = []
for i in range(len(array_occurance)):
    for j in range(len(array_occurance)):
        if (array_occurance[i][j] > 2):
            source.append(vocabulary[i])
            target.append(vocabulary[j])
            weight.append(array_occurance[i][j])
            if (vocabulary[i]) not in vocabulary_new:
                vocabulary_new.append(vocabulary[i])
            if (vocabulary[j]) not in vocabulary_new:
                vocabulary_new.append(vocabulary[j])
            

In [12]:
len(vocabulary_new)

26

In [13]:
df = pd.DataFrame(columns = ({"source", "target","weight"}))

In [14]:
df['source'] = source
df['target'] = target
df['weight'] = weight

In [15]:
df.to_csv(r'edgelist.csv')

In [16]:
df2 = pd.DataFrame(columns = ({"id", "role","salary"}))

In [17]:
df2['id'] = vocabulary_new
df2['role'] = 'word'
df2['salary'] = 0

In [18]:
df2.to_csv(r'nodelist.csv')